<a href="https://colab.research.google.com/github/JozeRamos/BigData/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Big Data Computing 2024 - Homework 2 - Deadline: December 22$^{nd}$, 11.59pm

# Instructions for the homework
  > Follow the instructions in the notebook carefully



## General instructions
  * Do NOT remove ANYTHING that is already in the notebook.
  * Before you start, show the line number for the code cells:
    * Go to Tools -> Settings -> Editor -> show line numbers
  * The group in the second Homework **MUST BE EXACTLY THE SAME** as in Homework 1
  * The software must be commented.
  * About this notebook:
    * Create a personal copy to be able to modify it: File > Save a copy in Drive
    * Rename the copied notebook `BDC2024-HW2.ipynb`


## Instructions for **code**:
  * DO NOT add code cells
  * DO NOT remove any code cells
  * Only edit the code cells containing the comment `#YOUR CODE STARTS HERE#`. Within those cells:
    * DO NOT remove ANYTHING that we have written (otherwise specified).
    * Add your code between line `#YOUR CODE STARTS HERE#` and line `#YOUR CODE ENDS HERE#`; **DO NOT REMOVE THESE COMMENTS**

> Ensure that the notebook can be faithfully reproduced by anyone (hint: pseudo random number generation).

> **If you need to set a random seed, set it to `1224`.**

[comment]: <> (#REMOVE_CELL#)


## Instructions for **text**:
* DO NOT add text cells
* DO NOT remove text cells
* DO NOT modify the text cells we have created.
* Only edit text cells starting with the comment ------------YOUR TEXT STARTS HERE------------. Within those cells:
  * Do NOT remove anything we have entered.
  * **Add your own text after the line ------------YOUR TEXT STARTS HERE------------; DO NOT REMOVE THIS TEXT**
  * For each cell, a specific delivery will be given. If you are given a maximum number of sentences to write, both the dot "." and the semicolon ";" will be considered sentence dividers.

[comment]: <> (#REMOVE_CELL#)


## *Evaluation*

The homework will be evaluated based on the following criteria:

1. **Code Quality**: correctness, readability (length, comments, unnecessary repetitions)
2. **Quality of Textual Responses**: correctness, clarity, etc.
3. **Quality of Produced Visualizations** (tables/figures/etc.): correctness, clarity, etc.
4. **Quality**, **quantity** and **diversity** of experiments conducted, **results** achieved, etc. \[if applicable; will be clearly indicated\]
5. **Timely Submission**: late submissions will result in significant point deductions
6. **Plagiarism**: copying code/text from colleagues or online sources will result in significant point deductions \[it will be clearly indicated if borrowing code from other sources is allowed\]

*You are expected to work in a principled way, being aware of what you are doing and why*

[comment]: <> (#REMOVE_CELL#)

## Group Composition
Write down the list of group members. The format should be Last Name, First Name, Student ID. Group members should be sorted alphabetically by surname and placed on different lines.
Example:

Becchetti, Luca, 123456

Siciliano, Federico, 987654

**Remember:** your group in the second homework must be the same as in the first one.



## Submission (read carefully)

1. This notebook is one of two parts of a Google Classroom assignment, hence it should be delivered using Google Classroom. Your delivery for Homework 2 will thus consist of two files (like for Homework 1): i) this notebook; ii) a pdf with your answers to the theoretical assignments.
2. Each notebook should be delivered *only once*, by the student whose last name comes first in alphabetical order in your group. So for example, if the group consists of Luca Becchetti and Federico Siciliano, Luca Becchetti (and *he only*) will deliver the homework (Federico Siciliano might well pay attention that Luca Becchetti actually does so within the mandatory deadline :-))
3. You can upload multiple files as part of the same assignment in Google Classroom. Please be sure that you only press the Classroom's "hand in" button ("Consegna" in Italian) after you have uploaded the final versions of this notebook and of the pdf containing your answers to the theoretical questions.

## Homework 2 (notebook)
We want to perform topic distillation/[keyword extraction](https://en.wikipedia.org/wiki/Keyword_extraction) using the techniques seen in class, in particular: 1) $k$-means preceeded by dimensionality reduction techniques and 2) SVD or PCA alone, as seen in class. You will use the [20 newsgroups dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html), in particular the ```['comp.graphics', 'rec.motorcycles', 'rec.sport.baseball', 'sci.space', 'talk.religion.misc']```categories.

##Software and dataset
Use the cell below to import **ALL** Python packages that you need for this homework.

Add them in this cell as you proceed with the implementation


In [ ]:
#REMOVE_OUTPUT#
!pip install --upgrade --no-cache-dir gdown
#YOUR CODE STARTS HERE#










#YOUR CODE ENDS HERE#
#THIS IS LINE 15#

Next, download the dataset as specified earlier

In [ ]:
#YOUR CODE STARTS HERE#







#YOUR CODE ENDS HERE#
#THIS IS LINE 10#

## Part 1 - keyword extraction/topic distillation using $k$-means
As a baseline, cluster documents using standard $k$-means. The number of clusters will be equal to the number of categories you downloaded. Quality will be measured as follows:
- Soundness of identified keywords: build word clouds of 20 most important keywords for each cluster
- Time efficiency
- Clustering quality with respect to true labels: [Adjusted Rand Index](https://en.wikipedia.org/wiki/Rand_index) (see scikit-learn documentation about)

This part must run within a maximum of **15** minutes. To meet this constraint, approximations can be made (using fewer documents, reducing dictionary size). However, these approximations **must be explicitly indicated** in the code.

### 1.1
If you need to, prepare here helper functions to run your computations.

In [ ]:
#YOUR CODE STARTS HERE#





































#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

###1.2
Text pre-processing and vectorization goes here

In [ ]:
#YOUR CODE STARTS HERE#

















#YOUR CODE ENDS HERE#
#THIS IS LINE 20#

### 1.1.3
Initiate time calculation for Part 1.

In [ ]:
#YOUR CODE STARTS HERE#







#YOUR CODE ENDS HERE#
#THIS IS LINE 10#

###1.1.4
Run $k$-means

In [ ]:
#YOUR CODE STARTS HERE#







#YOUR CODE ENDS HERE#
#THIS IS LINE 10#

Print Adjusted Rand Index and Time efficiency.

In [ ]:
#YOUR CODE STARTS HERE#







#YOUR CODE ENDS HERE#
#THIS IS LINE 10#

Print, for each cluster, wordcloud of 20 most important keyworkds.

In [ ]:
#YOUR CODE STARTS HERE#

















#YOUR CODE ENDS HERE#
#THIS IS LINE 20#

Summarize your results in 4 sentences at most.

------------YOUR TEXT STARTS HERE------------


## Part 2 - keyword extraction/topic distillation using Latent Semantic Analysis
We use Truncated SVD as an alternative baseline. In this case, we use the first $k$ components (singular vectors, with $k = 5$ in our case) of the Truncated SVD as descriptions of the latent topics and we identify the terms that are the most important for each component, as seen in class. Quality will be measured as follows:
- Soundness of identified keywords: build word clouds of 20 most important keywords for each cluster
- Time efficiency

This part must run within a maximum of **15** minutes. To meet this constraint, approximations can be made (using fewer documents, reducing dictionary size). However, these approximations **must be explicitly indicated** in the code.

### 2.1
If you need to, prepare here helper functions to run your computations.

In [ ]:
#YOUR CODE STARTS HERE#





































#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

###2.2
If you need to perform further text pre-processing and/or vectorization or redo it from scratch do it here. In this case, comment your code below to explain why you are doing this. Otherwise, leave the next cell empty.

In [ ]:
#YOUR CODE STARTS HERE#

















#YOUR CODE ENDS HERE#
#THIS IS LINE 20#

### 2.3
Initiate time calculation for Part 2.

In [ ]:
#YOUR CODE STARTS HERE#







#YOUR CODE ENDS HERE#
#THIS IS LINE 10#

###2.4
Perform Latent Semantic Analysis

In [ ]:
#YOUR CODE STARTS HERE#







#YOUR CODE ENDS HERE#
#THIS IS LINE 10#

Print Time efficiency.

In [ ]:
#YOUR CODE STARTS HERE#




#YOUR CODE ENDS HERE#
#THIS IS LINE 7#

Print, for each cluster, wordcloud of 20 most important keyworkds.

In [ ]:
#YOUR CODE STARTS HERE#

















#YOUR CODE ENDS HERE#
#THIS IS LINE 20#

Summarize your results in 4 sentences at most.

------------YOUR TEXT STARTS HERE------------


## Part 3 - keyword extraction/topic distillation using Truncated SVD + $k$-means
In this case, we i) first embed documents, i.e., project them onto the space spanned by the first $m$ singular vectors (for a suitable choice of $m$, see further) and then ii) we cluster the $m$-dimensional vectors thus obtained using $k$-means as before. Quality will be measured as follows:
- Clustering quality with respect to true labels: Adjusted Rand Index (see scikit-learn documentation about)
- Soundness of identified keywords: build word clouds of 20 most important keywords for each cluster
- Time efficiency

To determine the number $m$ of singular components to keep, use the elbow method to identify a first tentative value and then the adjusted Rand Index to find the right value. For example, if you plot total explained variance for $m$ ranging from $1$ to $100$ (or perhaps even 50), you are likely to see a clear elbow. Assume you identify 30 as a tentative value. To find the best choice for $m$, you can compute the adjusted Rand index of the corresponding clustering for all values of $m$ in a neighbourhood of $30$ (e.g., in the range $\{25,\ldots , 35\}$) and pick the value for which the index is maximum. This is just a simple heuristic, given to you as an example. If you think you found a better one you are free to use it, but you should briefly explain how it works. You are encouraged to use more principled heuristics proposed in the literature to choose $m$, in which case you should provide references about.

This part must run within a maximum of **20** minutes. To meet this constraint, approximations can be made (using fewer documents, reducing dictionary size). However, these approximations **must be explicitly indicated** in the code.

### 3.1
If you need to, prepare here helper functions to run your computations.

In [ ]:
#YOUR CODE STARTS HERE#





































#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

###3.2
If you need to perform further text pre-processing and/or vectorization or redo it from scratch do it here. In this case, comment your code below to explain why you are doing this. Otherwise, leave the next cell empty.

In [ ]:
#YOUR CODE STARTS HERE#

















#YOUR CODE ENDS HERE#
#THIS IS LINE 20#

### 3.3
Initiate time calculation for Part 3.

In [ ]:
#YOUR CODE STARTS HERE#







#YOUR CODE ENDS HERE#
#THIS IS LINE 10#

###3.4
Perform Truncated SVD and identify your best choice for the number $m$ of components to keep. Briefly explain the strategy you adopt to find $m$, using at most 5 lines in the text cell below. If you are simply following the heuristic I suggested above (which I did not test but might do a decent job), please just briefly state this.

------------YOUR TEXT STARTS HERE------------


Next, write your code to identify the best value of $m$.

In [ ]:
#YOUR CODE STARTS HERE#






























#YOUR CODE ENDS HERE#
#THIS IS LINE 33#

###3.5
Perform Truncated SVD (again if you did not save the results of the previous computation) with the number $m$ of components identified above. Cluster the projections of the documents onto the first $m$ components using $k$-means ($k = 5$ in our case).

In [ ]:
#YOUR CODE STARTS HERE#

















#YOUR CODE ENDS HERE#
#THIS IS LINE 20#

Print Time efficiency.

In [ ]:
#YOUR CODE STARTS HERE#




#YOUR CODE ENDS HERE#
#THIS IS LINE 7#

Print, for each cluster, wordcloud of 20 most important keyworkds.

In [ ]:
#YOUR CODE STARTS HERE#

















#YOUR CODE ENDS HERE#
#THIS IS LINE 20#

Summarize your results in 4 sentences at most.

------------YOUR TEXT STARTS HERE------------
